In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [0]:
batch_size = 64  
epochs = 100  
samples = 10000  
path = 'fra.txt'
latent_dim = 256  


In [0]:
inputtexts = []
targettexts = []
inputchars = set()
targetchars = set()
with open(path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(samples, len(lines) - 1)]:
    inputtext, targettext, _ = line.split('\t')
    
    targettext = '\t' + targettext + '\n'
    inputtexts.append(inputtext)
    targettexts.append(targettext)
    for char in inputtext:
        if char not in inputchars:
            inputchars.add(char)
    for char in targettext:
        if char not in targetchars:
            targetchars.add(char)

In [0]:
inputchars = sorted(list(inputchars))
targetchars = sorted(list(targetchars))
encodertokens = len(inputchars)
decodertokens = len(targetchars)
encoderseqlen = max([len(txt) for txt in inputtexts])
decoderseqlen = max([len(txt) for txt in targettexts])

In [5]:
print('No of samples:', len(inputtexts))
print('No of unique input tokens:', encodertokens)
print('No of unique output tokens:', decodertokens)
print('Max seq len for inputs:', encoderseqlen)
print('Max seq len for outputs:', decoderseqlen)


No of samples: 10000
No of unique input tokens: 71
No of unique output tokens: 93
Max seq len for inputs: 16
Max seq len for outputs: 59


In [0]:
inputtokenindex = dict(
    [(char, i) for i, char in enumerate(inputchars)])
targettokenindex = dict(
    [(char, i) for i, char in enumerate(targetchars)])

In [0]:
encoderinputdata = np.zeros(
    (len(inputtexts), encoderseqlen, encodertokens),
    dtype='float32')
decoderinputdata = np.zeros(
    (len(inputtexts), decoderseqlen, decodertokens),
    dtype='float32')
decodertargetdata = np.zeros(
    (len(inputtexts), decoderseqlen, decodertokens),
    dtype='float32')

In [0]:
for i, (inputtext, targettext) in enumerate(zip(inputtexts, targettexts)):
    for t, char in enumerate(inputtext):
        encoderinputdata[i, t, inputtokenindex[char]] = 1.
    encoderinputdata[i, t + 1:, inputtokenindex[' ']] = 1.
    for t, char in enumerate(targettext):
        decoderinputdata[i, t, targettokenindex[char]] = 1.
        if t > 0:
            
            decodertargetdata[i, t - 1, targettokenindex[char]] = 1.
    decoderinputdata[i, t + 1:, targettokenindex[' ']] = 1.
    decodertargetdata[i, t:, targettokenindex[' ']] = 1.

In [0]:
encoderinputs = Input(shape=(None, encodertokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoderinputs)
encoderstates = [state_h, state_c]

In [0]:
decoderinputs = Input(shape=(None, decodertokens))

decoderlstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoderoutputs, _, _ = decoderlstm(decoderinputs,
                                     initial_state=encoderstates)
decoderdense = Dense(decodertokens, activation='softmax')
decoderoutputs = decoderdense(decoderoutputs)

In [19]:

model = Model([encoderinputs, decoderinputs], decoderoutputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoderinputdata, decoderinputdata],decodertargetdata, batch_size=batch_size,
          
          epochs=epochs,
          validation_split=0.2)
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 17s 2ms/step - loss: 1.1783 - accuracy: 0.7243 - val_loss: 1.0487 - val_accuracy: 0.7123
Epoch 2/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.8574 - accuracy: 0.7693 - val_loss: 0.8549 - val_accuracy: 0.7645
Epoch 3/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.6916 - accuracy: 0.8056 - val_loss: 0.7245 - val_accuracy: 0.7883
Epoch 4/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.6126 - accuracy: 0.8241 - val_loss: 0.6765 - val_accuracy: 0.8024
Epoch 5/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.5579 - accuracy: 0.8366 - val_loss: 0.6271 - val_accuracy: 0.8164
Epoch 6/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.5191 - accuracy: 0.8471 - val_loss: 0.5871 - val_accuracy: 0.8270
Epoch 7/100
8000/8000 [==============================] - 15s 2ms/step - loss

In [0]:
encodermodel = Model(encoderinputs, encoderstates)

In [0]:
decoderstateinputh = Input(shape=(latent_dim,))
decoderstateinputc = Input(shape=(latent_dim,))
decoderstatesinputs = [decoderstateinputh, decoderstateinputc]
decoderoutputs, state_h, state_c = decoderlstm(
    decoderinputs, initial_state=decoderstatesinputs)
decoderstates = [state_h, state_c]
decoderoutputs = decoderdense(decoderoutputs)
decodermodel = Model(
    [decoderinputs] + decoderstatesinputs,
    [decoderoutputs] + decoderstates)


In [0]:

reverseinputcharindex = dict(
    (i, char) for char, i in inputtokenindex.items())
reversetargetcharindex = dict(
    (i, char) for char, i in targettokenindex.items())


In [0]:
def decodesequence(input_seq):
    statesvalue = encodermodel.predict(input_seq)

    targetseq = np.zeros((1, 1, decodertokens))
    targetseq[0, 0, targettokenindex['\t']] = 1.

    stopcondition = False
    decodedsentence = ''
    while not stopcondition:
        outputtokens, h, c = decodermodel.predict(
            [targetseq] + statesvalue)

        sampledtokenindex = np.argmax(outputtokens[0, -1, :])
        sampledchar = reversetargetcharindex[sampledtokenindex]
        decodedsentence += sampledchar


        if (sampledchar == '\n' or
           len(decodedsentence) > decoderseqlen):
            stopcondition = True

        targetseq = np.zeros((1, 1, decodertokens))
        targetseq[0, 0, sampledtokenindex] = 1.

        statesvalue = [h, c]
        
    return decodedsentence


In [29]:
for seq_index in range(100):
   
    input_seq = encoderinputdata[seq_index: seq_index + 1]
    decodedsentence = decodesequence(input_seq)
    print('-')
    print('Input sentence:', inputtexts[seq_index])
    print('Decoded sentence:', decodedsentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Bonjour !

-
Inpu